In [15]:
import pandas as pd
import re # "\n\ns.".replace("\n","").strip('.')
data = pd.read_csv("yelp_2013.csv")

In [16]:
data.head(3)

,Unnamed: 0,Unnamed: 0.1,text,stars,date,year
0,4,4,gute lage im stadtzentrum. shoppingmeile und s...,5,2013-11-20,2013
1,7,7,this is a fairly new property i think. it is a...,4,2013-07-07,2013
2,8,8,first time at this group of hotels. pretty new...,4,2013-04-27,2013


In [17]:
def get_data(path):
    token = {}
    data = pd.read_csv(path)
    last_data = {}
    last_data.setdefault("data",[])
    last_data.setdefault("label",[])
    
    for line in data.text:
        line = line.strip().split(" ")
        temp_line = [word.replace('\n','').strip('[.,""]').lower() for word in line if len(word)>0]
        last_data["data"].append(temp_line)
        for word in temp_line:
            token.setdefault(word,0)
            token[word] += 1
    for line in data.stars:
        last_data["label"].append(line)
    
    word2indx = {}
    word_list = []
    for key in token.keys():
        word_list.append(key)
        word2indx.setdefault(key, len(word_list)-1)
    return token, word2indx, word_list, last_data
        
tokens, word2indx, word_list, last_data= get_data("yelp_2013.csv")

In [18]:
token = {}
for line in data.text[:10]:
    temp_line = line.strip().split(" ")
    for word in temp_line:
        word = word.replace('\n','').strip('[.,""]')
        token.setdefault(word,0)
        token[word] += 1

In [168]:
# import spacy
# tokensizer = spacy.load('en')
# min(last_data['label'])

In [ ]:
# word_list
# import torch
da = []
for line in last_data['data']:
    temp = []
    for key in line:
        index = word2indx[key]
        temp.append(index)
    da.append(temp)

In [183]:
import numpy as np
da = np.load("da.npy")

In [6]:
def get_batch(das, labels, batch_size=128):
    m = len(das)
    index = np.random.permutation(m)
    das = np.array(das)[index]
    labels = np.array(labels)[index]
    n_batch = int(m / batch_size)
    for i in range(n_batch):
        yield das[i * batch_size : (i + 1) * batch_size], labels[i * batch_size : (i+1) * batch_size]

In [8]:
import keras
from keras.layers import Embedding, Input, GRU, Wrapper, Bidirectional, Dense, Lambda
from keras import Sequential
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.utils import  to_categorical
import keras.backend as K

In [21]:
data  = pad_sequences(da, maxlen=input_len, padding="post", truncating="post")
labels = to_categorical(np.array(last_data['label']) - 1)

def split_data(data, label):
    m = len(data)
    data = np.array(data)
    label = np.array(label)
    train_data = data[:int(m * 0.7)]
    train_label = label[:int(m * 0.7)]
    valid_data = data[int(m * 0.7) : int(m * 0.9)]
    valid_label = label[int(m * 0.7) : int(m * 0.9)]
    test_data = data[int(m * 0.9) :]
    test_label = label[int(m * 0.9) : ]
    return train_data, train_label, valid_data, valid_label, test_data, test_label
    
train_data, train_label, valid_data, valid_label, test_data, test_label =  split_data(data, labels)  

In [184]:
vocab_size = len(word2indx)
input_len = len(da[1])
print(vocab_size, input_len)
atten_size  = 10
hop = 3

687583 72


In [185]:
inputs = Input(shape=(72,))
embed = Embedding(input_dim=vocab_size + 1, input_length=input_len, output_dim=100)
x = embed(inputs)

In [190]:
rnn_model = Bidirectional(GRU(50, return_sequences=True))
rnn_result = rnn_model(x)
# b, n , zu
atten_1 = Dense(atten_size, activation="tanh", use_bias=False)(rnn_result) # b * n * da
atten_2 = Dense(hop, activation="softmax", use_bias=False)(atten_1) # b * n * r

def change_shape_1(tensors):
    atten_3 = K.reshape(tensors, [-1, 3, 72])
    atten_result = K.batch_dot(atten_3, rnn_result) 
    atten_flatten = K.reshape(atten_result, [-1, 3 * 2 * 50])
    return atten_flatten


atten_flatten = Lambda(change_shape_1)(atten_2)
atten_flatten.shape
#atten_3 = K.reshape(atten_2, [-1, 3, 72]) # b * r * n   // b * n * 2u
#atten_3.shape  # b * r * n 为最后的 attention 向量。

TensorShape([Dimension(None), Dimension(300)])

In [191]:
fc_model = Dense(10)
fc_model_re_1 = fc_model(atten_flatten)
fc_model_2 = Dense(5, activation="softmax")
out_put = fc_model_2(fc_model_re_1)
out_put.shape

TensorShape([Dimension(None), Dimension(5)])

In [192]:
model = Model(inputs, out_put)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 72)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 72, 100)           68758400  
_________________________________________________________________
bidirectional_5 (Bidirection (None, 72, 100)           45300     
_________________________________________________________________
dense_57 (Dense)             (None, 72, 10)            1000      
_________________________________________________________________
dense_58 (Dense)             (None, 72, 3)             30        
_________________________________________________________________
lambda_23 (Lambda)           (None, 300)               0         
_________________________________________________________________
dense_59 (Dense)             (None, 10)                3010      
__________

In [194]:
# 可自定义 损失函数，及使用 r * r 的正则化项目
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])

In [197]:
# just test !!
for iters in range(4):
    print("iter is {}".format(iters))
    model.fit(train_data[:10000], train_label[:10000], verbose=1, batch_size=512*4, epochs=10,
             validation_data=(valid_data[:1000], valid_label[:1000]))
    if iters % 2 ==0:
        score = model.evaluate(test_data, test_label)

iter is 0
iter is 1
iter is 2
iter is 3
